# Convert GPT text prompts to text prompts for LLMs on Amazon Bedrock

A [prompt](https://en.wikipedia.org/wiki/Prompt_engineering) to a [Large Language Model (LLM)](https://en.wikipedia.org/wiki/Large_language_model) is an instruction provided to it to produce an output.  There are many text [prompting techniques](https://www.promptingguide.ai/techniques).

This notebook will walk you through examples of converting [GPT](https://en.wikipedia.org/wiki/Generative_pre-trained_transformer) text prompts to text prompts for LLMs available on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html).

We will use the following process for prompt conversion,

* **Step 1:** Decompose the GPT text prompt into [prompt elements](https://www.promptingguide.ai/introduction/elements) using [Anthropic Claude 3](https://www.anthropic.com/news/claude-3-family) or [Anthropic Claude 3.5 Sonnet](https://www.anthropic.com/news/claude-3-5-sonnet) available on Amazon Bedrock.  These constructs are,
    * Context
    * Input Data
    * Output Indicator
    * Instruction
* **Step 2:** Parse these constructs using a Python script and create the text prompts for the various LLMs available on Amazon Bedrock.

We will cover the following types of prompts,

1. [Zero-Shot Prompting](https://www.promptingguide.ai/techniques/zeroshot)
2. [Few-Shot Prompting](https://www.promptingguide.ai/techniques/fewshot)
3. [Chain-of-Thought Prompting](https://www.promptingguide.ai/techniques/cot)
4. [Self-Consistency](https://www.promptingguide.ai/techniques/consistency)
5. [Generated Knowledge Prompting](https://www.promptingguide.ai/techniques/knowledge)

We will use [LangChain](https://www.langchain.com/) to simplify the process of constructing the prompts and interacting with the LLMs. In the process of working through this notebook, you will learn how to setup the Amazon Bedrock client environment, configure security permissions and use prompt templates in LangChain.

<div class="alert alert-block alert-warning">  
    <b>Disclaimer:</b> This prompt conversion process has been provided to accelerate your migration from GPT to LLMs on Amazon Bedrock. You must review the converted prompts for accuracy, optimize and test them before using in your projects.
</div>

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>This notebook uses text based models along with their versions that were available at the time of writing. Update these as required.</li>
        <li>At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions. Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.t3.medium</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 3</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Install required software libraries](#Install%20required%20software%20libraries)
    
    3. [Configure logging](#Configure%20logging)
        
        1. [System logs](#Configure%20system%20logs)
        
        2. [Application logs](#Configure%20application%20logs)
    
    4. [Organize imports](#Organize%20imports)
    
    5. [Create common objects](#Create%20common%20objects)
    
    6. [Enable model access in Amazon Bedrock](#Enable%20model%20access%20in%20Amazon%20Bedrock)
    
    7. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)
    
    8. [List the available Anthropic Claude 3 models](#List%20the%20available%20Anthropic%20Claude%203%20models)

 2. [Prompt conversion](#Prompt%20conversion)
 
    1. [Zero-shot Prompting](#Zero-shot%20Prompting)
    
    2. [Few-shot Prompting](#Few-shot%20Prompting)
    
    3. [Chain-of-Thought Prompting](#Chain-of-Thought%20Prompting)
    
    4. [Self-Consistency](#Self-Consistency)
    
    5. [Generated Knowledge Prompting](#Generated%20Knowledge%20Prompting)
 
 3. [Cleanup](#Cleanup)
 
 4. [Conclusion](#Conclusion)
 
 5. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
* [LangChain](https://www.langchain.com/)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.34.157
!pip install langchain==0.2.12
!pip install langchain-aws==0.1.16
!pip install langchain-community==0.2.11
!pip install sagemaker==2.228.0

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs <a id='Configure%20system%20logs'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

Run the following cell to print the name of the underlying notebook instance.

In [ ]:
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
import boto3
import langchain
import sagemaker
import sys
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))
logging.info("LangChain version : {}".format(langchain.__version__))

###  E. Create common objects <a id='Create%20common%20objects'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-warning">  
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions. In order to do this, this notebook will use the value specified in the environment variable named <mark>AMAZON_BEDROCK_REGION</mark>. If this is not specified, then the notebook will default to <mark>us-west-2 (Oregon)</mark> for Amazon Bedrock.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [ ]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

Create the rest of the common objects.

In [ ]:
# Create the Amazon Bedrock client
bedrock_client = boto3.client("bedrock", region_name = br_region, endpoint_url = "https://bedrock.{}.amazonaws.com"
                              .format(br_region), config = my_boto3_config)

# Create the Amazon Bedrock runtime client
bedrock_rt_client = boto3.client("bedrock-runtime", region_name = br_region, config = my_boto3_config)

# Specify the directories for the source prompts
gpt_source_prompts_dir = os.path.join(os.getcwd(), "source_prompts", "gpt")

# Specify the directories for the target prompts
amazon_titan_text_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "amazon_titan_text")
anthropic_claude_2_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "anthropic_claude_2")
anthropic_claude_3_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "anthropic_claude_3")
anthropic_claude_3_5_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "anthropic_claude_3.5")
ai21_labs_jurassic_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "ai21_labs_jurassic")
cohere_command_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "cohere_command")
meta_llama_2_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "meta_llama_2")
meta_llama_3_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "meta_llama_3")
meta_llama_3_1_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "meta_llama_3.1")
mistral_ai_target_prompts_dir = os.path.join(os.getcwd(), "target_prompts", "mistral_ai")

# Create the target prompt directories if they doesn't exist
os.makedirs(amazon_titan_text_target_prompts_dir, exist_ok = True)
os.makedirs(anthropic_claude_2_target_prompts_dir, exist_ok = True)
os.makedirs(anthropic_claude_3_target_prompts_dir, exist_ok = True)
os.makedirs(anthropic_claude_3_5_target_prompts_dir, exist_ok = True)
os.makedirs(ai21_labs_jurassic_target_prompts_dir, exist_ok = True)
os.makedirs(cohere_command_target_prompts_dir, exist_ok = True)
os.makedirs(meta_llama_2_target_prompts_dir, exist_ok = True)
os.makedirs(meta_llama_3_target_prompts_dir, exist_ok = True)
os.makedirs(meta_llama_3_1_target_prompts_dir, exist_ok = True)
os.makedirs(mistral_ai_target_prompts_dir, exist_ok = True)

# Specify the name of the prompt decomposition template file name
prompt_templates_dir = os.path.join(os.getcwd(), "prompt_templates")
system_prompt_decomposition_template = 'Claude_3.x_system_prompt_template_for_GPT_decomposition.txt'
user_prompt_decomposition_template = 'Claude_3.x_user_prompt_template_for_GPT_decomposition.txt'

###  F. Enable model access in Amazon Bedrock <a id ='Enable%20model%20access%20in%20Amazon%20Bedrock'> </a>

<div class="alert alert-block alert-danger">
    <b>Note:</b> Before proceeding further with this notebook, you must enable access to the Anthropic Claude 3 models on Amazon Bedrock by following the instructions <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">here</a>. You need to submit the use case details. Otherwise, you will get an authorization error.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> You will have to do this manually after reading the End User License Agreement (EULA) for each of the models that you want to enable. Unless you explicitly disable it, this is a one-time setup for each model in an AWS account.
</div>

Run the following cell to print the Amazon Bedrock model access page URL for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock model access page URL
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess"
             .format(br_region, br_region))

<div class="alert alert-block alert-info">  
<b>Note:</b> For running this notebook, you need access to only the Anthropic Claude 3 models. However, if you want to run the converted prompts against the respective models on Amazon Bedrock, then, you need to enable access to those models as well. 
</div>

After enabling all the required model access, you can try out the converted prompts on the [Amazon Bedrock playgrounds](https://docs.aws.amazon.com/bedrock/latest/userguide/playgrounds.html). Run the following cell to print the URL for the [text playground](https://docs.aws.amazon.com/bedrock/latest/userguide/playgrounds.html#text-playground) for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock text model playground URL
logging.info("Amazon Bedrock text model playground - https://{}.console.aws.amazon.com/bedrock/home?region={}#/text-playground"
             .format(br_region, br_region))

###  G. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell.

This IAM role should have the following permissions,

1. Full access to invoke Large Language Models (LLMs) on Amazon Bedrock.
2. Access to write to Amazon CloudWatch Logs.

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

###  H. List the available Anthropic Claude 3 models <a id='List%20the%20available%20Anthropic%20Claude%203%20models'> </a>

Running the following cell will list all Anthropic Claude 3 and 3.5 LLMs on Amazon Bedrock that are offered through the On-Demand throughput pricing model. This will help you pick the model-ids that you will use further down in this notebook.

For more information on this, refer [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns).

In [ ]:
# List all the available Anthropic Claude 3 and 3.5 LLMs in Amazon Bedrock with On-Demand throughput pricing
models_info = ''
response = bedrock_client.list_foundation_models(byProvider = "Anthropic",
                                                 byInferenceType = "ON_DEMAND")
model_summaries = response["modelSummaries"]
models_info = models_info + "\n"
models_info = models_info + "-".ljust(125, "-") + "\n"
models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format("Provider Name", "Model Name", "Input Modalities",
                                                          "Output Modalities", "Model Id")
models_info = models_info + "-".ljust(125, "-")
for model_summary in model_summaries:
    # Check for Claude 3 and 3.5 LLMs and process
    if model_summary["modelName"] in ("Claude 3 Sonnet", "Claude 3 Haiku", "Claude 3 Opus", "Claude 3.5 Sonnet"):
        models_info = models_info + "\n"
        models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format(model_summary["providerName"],
                                                                                model_summary["modelName"],
                                                                                "|".join(model_summary["inputModalities"]),
                                                                                "|".join(model_summary["outputModalities"]),
                                                                                model_summary["modelId"])
models_info = models_info + "-".ljust(125, "-") + "\n"
logging.info("Displaying available Anthropic Claude 3 and 3.5 models in the '{}' Region:".format(br_region) + models_info)

## 2. Prompt conversion <a id ='Prompt%20conversion'> </a>

We will use the following process for prompt conversion,

* **Step 1:** Decompose the GPT text prompt into [prompt elements](https://www.promptingguide.ai/introduction/elements) using [Anthropic Claude 3](https://www.anthropic.com/news/claude-3-family) or [Anthropic Claude 3.5 Sonnet](https://www.anthropic.com/news/claude-3-5-sonnet) available on Amazon Bedrock.  These constructs are,
    * Context
    * Input Data
    * Output Indicator
    * Instruction
* **Step 2:** Parse these constructs using a Python script and create the text prompts for the various LLMs available on Amazon Bedrock.

In [ ]:
# Specify the Anthropic Claude 3 or 3.5 model-id
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
#model_id = "anthropic.claude-3-haiku-20240307-v1:0"
#model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
#model_id = "anthropic.claude-3-opus-20240229-v1:0"

### A. Zero-shot Prompting <a id ='Zero-shot%20Prompting'> </a>

For info on Zero-shot Prompting, refer [here](https://www.promptingguide.ai/techniques/zeroshot).

In [ ]:
# Specify the source prompt file name
source_prompt_file_name = 'GPT_zero_shot_prompt_1.txt'

View the GPT prompt content by running the following cell.

In [ ]:
!cat {gpt_source_prompts_dir}/{source_prompt_file_name}

**Step 1:** Take the GPT prompt and decompose it into prompt constructs.

In [ ]:
# Read the data from the file that contains
# the source (GPT) prompt (./scripts/helper_functions.py)
source_prompt = read_file(gpt_source_prompts_dir, source_prompt_file_name)

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
# Note that the temperature will be set to 0 in the helper function
decomposed_source_prompt = process_prompt(model_id,
                                          bedrock_rt_client,
                                          prompt_templates_dir,
                                          system_prompt_decomposition_template,
                                          user_prompt_decomposition_template,
                                          source_prompt)

**Step 2:** Parse the decomposed prompt constructs and create the prompts for LLMs on Amazon Bedrock.

In [ ]:
# Using the helper functions in (./scripts/helper_functions.py)

## Amazon Titan Text
convert_decomposed_prompt('Amazon Titan Text', amazon_titan_text_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 2
convert_decomposed_prompt('Anthropic Claude 2', anthropic_claude_2_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 3 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3', anthropic_claude_3_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 3.5 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3.5', anthropic_claude_3_5_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## AI21 Labs Jurassic
convert_decomposed_prompt('AI21 Labs Jurassic', ai21_labs_jurassic_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Cohere Command
convert_decomposed_prompt('Cohere Command', cohere_command_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 2
convert_decomposed_prompt('Meta LLAMA 2', meta_llama_2_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 3
convert_decomposed_prompt('Meta LLAMA 3', meta_llama_3_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 3.1
convert_decomposed_prompt('Meta LLAMA 3.1', meta_llama_3_1_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

## Mistral AI
convert_decomposed_prompt('Mistral AI', mistral_ai_target_prompts_dir, 'zero_shot_prompt', decomposed_source_prompt)

### B. Few-shot Prompting <a id ='Few-shot%20Prompting'> </a>

For info on Few-shot Prompting, refer [here](https://www.promptingguide.ai/techniques/fewshot).

In [ ]:
# Specify the source prompt file name
#source_prompt_file_name = 'GPT_few_shot_prompt_1.txt'
source_prompt_file_name = 'GPT_few_shot_prompt_2.txt'

View the GPT prompt content by running the following cell.

In [ ]:
!cat {gpt_source_prompts_dir}/{source_prompt_file_name}

**Step 1:** Take the GPT prompt and decompose it into prompt constructs.

In [ ]:
# Read the data from the file that contains
# the source (GPT) prompt (./scripts/helper_functions.py)
source_prompt = read_file(gpt_source_prompts_dir, source_prompt_file_name)

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
# Note that the temperature will be set to 0 in the helper function
decomposed_source_prompt = process_prompt(model_id,
                                          bedrock_rt_client,
                                          prompt_templates_dir,
                                          system_prompt_decomposition_template,
                                          user_prompt_decomposition_template,
                                          source_prompt)

**Step 2:** Parse the decomposed prompt constructs and create the prompts for LLMs on Amazon Bedrock.

In [ ]:
# Using the helper functions in (./scripts/helper_functions.py)

## Amazon Titan Text
convert_decomposed_prompt('Amazon Titan Text', amazon_titan_text_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 2
convert_decomposed_prompt('Anthropic Claude 2', anthropic_claude_2_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 3 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3', anthropic_claude_3_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Anthropic Claude 3.5 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3.5', anthropic_claude_3_5_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## AI21 Labs Jurassic
convert_decomposed_prompt('AI21 Labs Jurassic', ai21_labs_jurassic_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Cohere Command
convert_decomposed_prompt('Cohere Command', cohere_command_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 2
convert_decomposed_prompt('Meta LLAMA 2', meta_llama_2_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 3
convert_decomposed_prompt('Meta LLAMA 3', meta_llama_3_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Meta LLAMA 3.1
convert_decomposed_prompt('Meta LLAMA 3.1', meta_llama_3_1_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

## Mistral AI
convert_decomposed_prompt('Mistral AI', mistral_ai_target_prompts_dir, 'few_shot_prompt', decomposed_source_prompt)

### C. Chain-of-Thought Prompting <a id ='Chain-of-Thought%20Prompting'> </a>

For info on Chain-of-Thought Prompting, refer [here](https://www.promptingguide.ai/techniques/cot).

In [ ]:
# Specify the source prompt file name
source_prompt_file_name = 'GPT_chain_of_thought_prompt_1.txt'

View the GPT prompt content by running the following cell.

In [ ]:
!cat {gpt_source_prompts_dir}/{source_prompt_file_name}

**Step 1:** Take the GPT prompt and decompose it into prompt constructs.

In [ ]:
# Read the data from the file that contains
# the source (GPT) prompt (./scripts/helper_functions.py)
source_prompt = read_file(gpt_source_prompts_dir, source_prompt_file_name)

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
# Note that the temperature will be set to 0 in the helper function
decomposed_source_prompt = process_prompt(model_id,
                                          bedrock_rt_client,
                                          prompt_templates_dir,
                                          system_prompt_decomposition_template,
                                          user_prompt_decomposition_template,
                                          source_prompt)

**Step 2:** Parse the decomposed prompt constructs and create the prompts for LLMs on Amazon Bedrock.

In [ ]:
# Using the helper functions in (./scripts/helper_functions.py)

## Amazon Titan Text
convert_decomposed_prompt('Amazon Titan Text', amazon_titan_text_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Anthropic Claude 2
convert_decomposed_prompt('Anthropic Claude 2', anthropic_claude_2_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Anthropic Claude 3 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3', anthropic_claude_3_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Anthropic Claude 3.5 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3.5', anthropic_claude_3_5_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## AI21 Labs Jurassic
convert_decomposed_prompt('AI21 Labs Jurassic', ai21_labs_jurassic_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Cohere Command
convert_decomposed_prompt('Cohere Command', cohere_command_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Meta LLAMA 2
convert_decomposed_prompt('Meta LLAMA 2', meta_llama_2_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Meta LLAMA 3
convert_decomposed_prompt('Meta LLAMA 3', meta_llama_3_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Meta LLAMA 3.1
convert_decomposed_prompt('Meta LLAMA 3.1', meta_llama_3_1_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

## Mistral AI
convert_decomposed_prompt('Mistral AI', mistral_ai_target_prompts_dir, 'chain_of_thought_prompt', decomposed_source_prompt)

### D. Self-Consistency <a id ='Self-Consistency'> </a>

For info on Self-Consistency, refer [here](https://www.promptingguide.ai/techniques/consistency).

In [ ]:
# Specify the source prompt file name
source_prompt_file_name = 'GPT_self_consistency_prompt_1.txt'

View the GPT prompt content by running the following cell.

In [ ]:
!cat {gpt_source_prompts_dir}/{source_prompt_file_name}

**Step 1:** Take the GPT prompt and decompose it into prompt constructs.

In [ ]:
# Read the data from the file that contains
# the source (GPT) prompt (./scripts/helper_functions.py)
source_prompt = read_file(gpt_source_prompts_dir, source_prompt_file_name)

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
# Note that the temperature will be set to 0 in the helper function
decomposed_source_prompt = process_prompt(model_id,
                                          bedrock_rt_client,
                                          prompt_templates_dir,
                                          system_prompt_decomposition_template,
                                          user_prompt_decomposition_template,
                                          source_prompt)

**Step 2:** Parse the decomposed prompt constructs and create the prompts for LLMs on Amazon Bedrock.

In [ ]:
# Using the helper functions in (./scripts/helper_functions.py)

## Amazon Titan Text
convert_decomposed_prompt('Amazon Titan Text', amazon_titan_text_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Anthropic Claude 2
convert_decomposed_prompt('Anthropic Claude 2', anthropic_claude_2_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Anthropic Claude 3 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3', anthropic_claude_3_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Anthropic Claude 3.5 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3.5', anthropic_claude_3_5_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## AI21 Labs Jurassic
convert_decomposed_prompt('AI21 Labs Jurassic', ai21_labs_jurassic_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Cohere Command
convert_decomposed_prompt('Cohere Command', cohere_command_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Meta LLAMA 2
convert_decomposed_prompt('Meta LLAMA 2', meta_llama_2_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Meta LLAMA 3
convert_decomposed_prompt('Meta LLAMA 3', meta_llama_3_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Meta LLAMA 3.1
convert_decomposed_prompt('Meta LLAMA 3.1', meta_llama_3_1_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

## Mistral AI
convert_decomposed_prompt('Mistral AI', mistral_ai_target_prompts_dir, 'self_consistency_prompt', decomposed_source_prompt)

### E. Generated Knowledge Prompting <a id ='Generated%20Knowledge%20Prompting'> </a>

For info on Generated Knowledge Prompting, refer [here](https://www.promptingguide.ai/techniques/knowledge).

In [ ]:
# Specify the source prompt file name
source_prompt_file_name = 'GPT_general_knowledge_prompt_1.txt'

View the GPT prompt content by running the following cell.

In [ ]:
!cat {gpt_source_prompts_dir}/{source_prompt_file_name}

**Step 1:** Take the GPT prompt and decompose it into prompt constructs.

In [ ]:
# Read the data from the file that contains
# the source (GPT) prompt (./scripts/helper_functions.py)
source_prompt = read_file(gpt_source_prompts_dir, source_prompt_file_name)

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
# Note that the temperature will be set to 0 in the helper function
decomposed_source_prompt = process_prompt(model_id,
                                          bedrock_rt_client,
                                          prompt_templates_dir,
                                          system_prompt_decomposition_template,
                                          user_prompt_decomposition_template,
                                          source_prompt)

**Step 2:** Parse the decomposed prompt constructs and create the prompts for LLMs on Amazon Bedrock.

In [ ]:
# Using the helper functions in (./scripts/helper_functions.py)

## Amazon Titan Text
convert_decomposed_prompt('Amazon Titan Text', amazon_titan_text_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Anthropic Claude 2
convert_decomposed_prompt('Anthropic Claude 2', anthropic_claude_2_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Anthropic Claude 3 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3', anthropic_claude_3_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Anthropic Claude 3.5 (Messages API)
convert_decomposed_prompt('Anthropic Claude 3.5', anthropic_claude_3_5_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## AI21 Labs Jurassic
convert_decomposed_prompt('AI21 Labs Jurassic', ai21_labs_jurassic_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Cohere Command
convert_decomposed_prompt('Cohere Command', cohere_command_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Meta LLAMA 2
convert_decomposed_prompt('Meta LLAMA 2', meta_llama_2_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Meta LLAMA 3
convert_decomposed_prompt('Meta LLAMA 3', meta_llama_3_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Meta LLAMA 3.1
convert_decomposed_prompt('Meta LLAMA 3.1', meta_llama_3_1_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## Mistral AI
convert_decomposed_prompt('Mistral AI', mistral_ai_target_prompts_dir, 'general_knowledge_prompt', decomposed_source_prompt)

## 3. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete AWS resources that are no longer required.  This will help you avoid incurring unncessary costs.

The minimum cleanup required for this notebook would be shutdown the instance on which this notebook is running.

## 4. Conclusion <a id='Conclusion'></a>

We have now seen how to convert GPT text prompts to text prompts for LLMs available on [Amazon Bedrock](https://aws.amazon.com/bedrock/). Through this, you were able to learn how the prompts are different for different LLMs and how to use prompt templates.

## 5. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Will Amazon Bedrock capture and store my data?**

Amazon Bedrock doesn't use your prompts and continuations to train any AWS models or distribute them to third parties. Your training data isn't used to train the base Amazon Titan models or distributed to third parties. Other usage data, such as usage timestamps, logged account IDs, and other information logged by the service, is also not used to train the models.

Amazon Bedrock uses the fine tuning data you provide only for fine tuning an Amazon Titan model. Amazon Bedrock doesn't use fine tuning data for any other purpose, such as training base foundation models.

Each model provider has an escrow account that they upload their models to. The Amazon Bedrock inference account has permissions to call these models, but the escrow accounts themselves don't have outbound permissions to Amazon Bedrock accounts. Additionally, model providers don't have access to Amazon Bedrock logs or access to customer prompts and continuations.

Amazon Bedrock doesn’t store or log your data in its service logs.

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: What is the difference between On-demand and Provisioned Throughput in Amazon Bedrock?**

With the On-Demand mode, you only pay for what you use, with no time-based term commitments. For text generation models, you are charged for every input token processed and every output token generated. For embeddings models, you are charged for every input token processed. A token is comprised of a few characters and refers to the basic unit that a model learns to understand user input and prompt to generate results. For image generation models, you are charged for every image generated.

With the Provisioned Throughput mode, you can purchase model units for a specific base or custom model. The Provisioned Throughput mode is primarily designed for large consistent inference workloads that need guaranteed throughput. Custom models can only be accessed using Provisioned Throughput. A model unit provides a certain throughput, which is measured by the maximum number of input or output tokens processed per minute. With this Provisioned Throughput pricing, charged by the hour, you have the flexibility to choose between 1-month or 6-month commitment terms.

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find resources for prompt engineering?**

[Prompt Engineering Guide](https://www.promptingguide.ai/).

**Q: Where can I find prompt guides for the various LLMs available in Amazon Bedrock?**

* Amazon Titan models, refer [here](https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf).
* Anthropic Claude models, refer [here](https://docs.anthropic.com/claude/docs/constructing-a-prompt).
* AI21 Labs Jurassic models, refer [here](https://docs.ai21.com/docs/prompt-engineering).
* Cohere Command models, refer [here](https://txt.cohere.com/constructing-prompts/).
* Meta LLAMA 2 models, refer [here](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-2).
* Meta LLAMA 3 models, refer [here](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3).
* Mistral AI models, refer [here](https://docs.mistral.ai/guides/prompting-capabilities/).

**Q: Is LangChain mandatory to use Amazon Bedrock?**

No. You can interact with Amazon Bedrock using the [Bedrock API](https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html) or language-specific [AWS SDKs](https://aws.amazon.com/developer/tools/). 

**Q: How do I get started with LangChain?**

Go [here](https://python.langchain.com/docs/get_started/introduction).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).